# Tiny Dataset notebook

In [3]:
# import sqlite3
# import pandas as pd
# import os
# import openai
# from freeplay import Freeplay 
# from dotenv import load_dotenv
# 
# load_dotenv()
# 
# openai.api_key = os.environ.get("OPENAI_API_KEY")
# OPEN_AI_KEY = os.environ.get("OPENAI_API_KEY")
# FREEPLAY_API_KEY = os.environ.get("FREEPLAY_API_KEY")
# 
# fp_client = Freeplay(
#     provider_config=ProviderConfig(openai=OpenAIConfig(OPENAI_API_KEY)),
#     freeplay_api_key=FREEPLAY_API_KEY,
#     api_base=f'https://8thlight.freeplay.ai/api'
# )
# 
# prompt_vars = {"keyA": "valueA"}
# completion = fp_client.get_cpmpletion(project_id='AI_Cohort_1_Group_3_Explorations',
#                                       template_name="Evaluation of Summary Capabilities",
#                                       variables=prompt_vars)
# 
# 
# base_path = f"{os.getcwd()}"
# 
# file_path = os.path.join(os.path.dirname(base_path), 'data/ctg-studies-tiny.db')
# 
# conn = sqlite3.connect(file_path)
# 
# query = "SELECT * FROM tiny_dataset"
# df = pd.read_sql_query(query, conn)
# # data_summary = df.describe().to_string() use a summary of the data to pass to LM
# data = df.head()
# 
# 
# messages = 
# response = openai.chat.completions.create(
#     model="gpt-4-turbo",
#     messages=messages
# )
# 
# # Keeps the conversational context intact
# assistant_response = response.choices[0].message.content
# print(assistant_response)
# messages.append({
#     "role": "assistant",
#     "content": assistant_response
# })


The data provided is a summary of clinical studies with various attributes such as study ID, completion date, study type, eligibility, and demographic details among others. Here is a summarized overview of five specific clinical studies within the dataset:

1. **NCT00436514** (Sequence 286365)
   - **Completion**: July 2009 (Actual)
   - **Summary**: Focused on colorectal cancer, being a frequent cause of death particularly in Europe.
   - **Description**: Not provided.
   - **Condition**: Colorectal Cancer
   - **Study Type**: Interventional
   - **Eligibility**: Adults aged 18 to 80 years.
   - **Volunteers**: Not accepting healthy volunteers.
   - **Sex**: All
   - **Age Range**: 18 to 80 years
   - **References**: Includes a derived citation (PMID: 21039676).

2. **NCT02827617** (Sequence 150334)
   - **Completion**: Expected by December 31, 2024 (Estimated)
   - **Summary**: Aims to identify new prognostic factors for chronic lymphocytic leukemia.
   - **Description**: Focuses on 

### Continue the conversation
Replace the content of the `new_user_message` variable with different prompts to continue the conversation.

In [ ]:
# new_user_message = "Is there anything interesting I should know bout this data?"
# 
# messages.append({
#     "role": "user",
#     "content": new_user_message
# })
# 
# response = openai.chat.completions.create(
#     model="gpt-4-turbo",
#     messages=messages
# )
# assistant_response = response.choices[0].message.content
# print(assistant_response)
# messages.append({
#     "role": "assistant",
#     "content": assistant_response
# })

In [113]:
import time
import os
import openai
from freeplay import Freeplay, RecordPayload, CallInfo, ResponseInfo, SessionInfo, CallInfo
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.environ.get("OPENAI_API_KEY")
OPEN_AI_KEY = os.environ.get("OPENAI_API_KEY")
FREEPLAY_API_KEY = os.environ.get("FREEPLAY_API_KEY")


# create your a freeplay client object
fpClient = Freeplay(
    freeplay_api_key=os.getenv("FREEPLAY_API_KEY"),
    api_base="https://8thlight.freeplay.ai/api"
)

## PROMPT FETCH ##
# set the prompt variables
prompt_vars = {"keyA": "valueA"}
# get a formatted prompt
formatted_prompt = fpClient.prompts.get_formatted(project_id="473ef06b-2883-41da-85dd-9ccec95c95c6",
                                                  template_name="Evaluation of Summary Capabilities",
                                                  environment="latest",
                                                  variables=prompt_vars)
# print(getattr(formatted_prompt, "messages"))
assistant_messages = [message for message in formatted_prompt.messages if message['role'] == "assistant"]
# print(assistant_messages)

## LLM CALL ##
# Make an LLM call to your provider of choice
# start = time.time()
# chat_response =  openai.chat.completions.create(
#     model=formatted_prompt.prompt_info.model,
#     messages=formatted_prompt.messages,
#     **formatted_prompt.prompt_info.model_parameters
# )
# end = time.time()

# # add the response to your message set
# all_messages = formatted_prompt.all_messages(
#     {'role': chat_response.choices[0].message.role,
#      'content': chat_response.choices[0].message.content}
# )

# ## RECORD ##
# # create a session
# session = fpClient.sessions.create()

# # build the record payload
# payload = RecordPayload(
#     all_messages=all_messages,
#     inputs=prompt_vars,
#     session_info=session,
#     prompt_info=formatted_prompt.prompt_info,
#     call_info=CallInfo.from_prompt_info(formatted_prompt.prompt_info, start_time=start, end_time=end),
#     response_info=ResponseInfo(
#         is_complete=chat_response.choices[0].finish_reason == 'stop'
#     )
# )
# # record the LLM interaction
# fpClient.recordings.create(payload)

# 
file_path = '/Users/kroy/ai-cohort/ai_cohort/data/ctg-studies-tiny.csv'

test_file = openai.files.create(
    file=open(file_path, "rb"),
    purpose='assistants'
)

my_assistant = openai.beta.assistants.create(
    model="gpt-4o",
    instructions=getattr(formatted_prompt, "system_content"),
    name="Data Analysis Chatbot",
    tools=[{"type": "code_interpreter"}],
    tool_resources={
        "code_interpreter": {
          "file_ids": [test_file.id]
        }
    }
)

my_thread = openai.beta.threads.create()

my_thread_message = openai.beta.threads.messages.create(
  thread_id=my_thread.id,
  role="user",
  content=assistant_messages[0]["content"],
  attachments=[
    {
      "file_id": test_file.id,
      "tools": [
        { "type": "code_interpreter" }
      ]
    }
  ]
)

my_run = openai.beta.threads.runs.create(
  thread_id=my_thread.id,
  assistant_id=my_assistant.id,
)

keep_retrieving_run = openai.beta.threads.runs.retrieve(
    thread_id=my_thread.id,
    run_id=my_run.id
)

run_steps = openai.beta.threads.runs.steps.list(
  thread_id=my_thread.id,
  run_id=my_run.id
)

# if response.status == 'completed':
#                     print('Run completed!')
#                     respmsg = self.openai.beta.threads.messages.list(threadId)
#                     print(respmsg.data[0].content[0].text.value)

#                     outgoingMessages = respmsg.data[0].content[0].text.value

# respmsg = openai.beta.threads.messages.list(my_thread.id)
# print(respmsg)
# outgoingMessages = respmsg.data[0].content[0].text.value
# print(run_steps)
def pollAnswers(threadId, runId):
        completed = False
        start = time.time()
        while not completed:
            try:
                response = openai.beta.threads.runs.retrieve(
                    thread_id=threadId,
                    run_id=runId
                )

                # bstack.emit(
                #     'status.update',
                #     {
                #         'status': response.status,
                #         'id': response.id,
                #         'created_at': response.created_at,
                #         'completed_at': response.completed_at
                #     }
                # )

                if response.status == 'completed':
                    end = time.time()
                    print('Run completed!')
                    respmsg = openai.beta.threads.messages.list(threadId)
                    print(respmsg.data[0].content[0].text.value)

                    # Make an LLM call to your provider of choice
                    # chat_response =  openai.chat.completions.create(
                    #     model=formatted_prompt.prompt_info.model,
                    #     messages=formatted_prompt.messages,
                    #     **formatted_prompt.prompt_info.model_parameters
                    # )
                    
                    
                    # # add the response to your message set
                    all_messages = formatted_prompt.all_messages(
                        {'role': respmsg.data[0].role,
                         'content': respmsg.data[0].content[0].text.value}
                    )
                    
                    # ## RECORD ##
                    # # create a session
                    session = fpClient.sessions.create()
                    
                    # # build the record payload
                    payload = RecordPayload(
                        all_messages=all_messages,
                        inputs=prompt_vars,
                        session_info=session,
                        prompt_info=formatted_prompt.prompt_info,
                        call_info=CallInfo.from_prompt_info(formatted_prompt.prompt_info, start_time=start, end_time=end),
                        response_info=ResponseInfo(
                            is_complete=chat_response.choices[0].finish_reason == 'stop'
                        )
                    )
                    # record the LLM interaction
                    fpClient.recordings.create(payload)
                    

                    outgoingMessages = respmsg.data[0].content[0].text.value
                    completed = True
                    # bstack.emit('chat.message', {'message': outgoingMessages})
                    break

                if response.status == 'expired':
                    print('Task expired!')
                    respmsg = openai.beta.threads.messages.list(threadId)
                    print(respmsg.data[0].content[0].text.value)

                    outgoingMessages = respmsg.data[0].content[0].text.value
                    completed = True
                    # bstack.emit('chat.message', {'message': outgoingMessages})
                    break

                if response.status == 'requires_action':
                    print('Run Action Required!')

                    tool_outputs = []
                    for ra in response.required_action.submit_tool_outputs.tool_calls:
                        if ra.type == 'function' and ra.function.name in functions:
                            print(f'Running {ra.function.name} with arguments {ra.function.arguments}')
                            ret = functions[ra.function.name](ra.function.arguments)
                            print('Returned', ret)
                            tool_outputs.append({
                                'tool_call_id': ra.id,
                                'output': json.dumps(ret)
                            })
                        else:
                            tool_outputs.append({
                                'tool_call_id': ra.id,
                                'output': 'There is a problem, function not found!'
                            })

                    run = openai.beta.threads.runs.submitToolOutputs(
                        threadId,
                        runId,
                        {'tool_outputs': tool_outputs}
                    )

                if response.status in ('failed', 'cancelled', 'expired', 'cancelling'):
                    print(f'Completed {response.status}')
                    completed = True
                    break

                if response.status in ('in_progress', 'queued'):
                    print(f'Run {response.status}')
                    time.sleep(2)  # Poll every 2 seconds

            except Exception as e:
                print('Error in polling (retrying):', str(e))
                # Handle the exception or retry logic as necessary

pollAnswers(my_thread.id, my_run.id)

Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run in_progress
Run completed!
Here are the first few entries from the `summary` column:

1. **Row 0**: "Colorectal cancer is one of the most frequent ..."
2. **Row 1**: "The general aim of the project is the identifi..."
3. **Row 2**: "The purpose of this clinical project is to eva..."
4. **Row 3**: "The purpose of this study is to determine what..."
5. **Row 4**: "This observational study will evaluate the cli..."

If you need a more detailed review or have specific areas you want to expand on, please let me know!


Run(id='run_CN6GQDGwXgu3e9iKflUQtZVy', assistant_id='asst_uj4I5CHPcB2VRrBAlE9pjq48', cancelled_at=None, completed_at=None, created_at=1729638936, expires_at=1729639536, failed_at=None, incomplete_details=None, instructions='You are a clinical trial expert', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_817xEYjwWKciLwyoRm1qoSD4', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


In [ ]:
OPEN_AI_KEY = os.environ.get("OPENAI_API_KEY")
service = OpenAIService(OPEN_AI_KEY)
